<a href="https://colab.research.google.com/github/dbetm/DeepLearningLifeSciences/blob/main/Binding_free_energy_prediction_using_PDBBind_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binding free energy prediction using PDBBind dataset

PDB stands for [Protein Data Bank](https://www.rcsb.org/).

**Problem**: Predicting the binding affinity of protein–ligand
systems.

The binding free energy is described as a sum of the intermolecular interactions between the ligand and the protein and the internal steric energy of the ligand.

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e
!pip install --pre deepchem
import deepchem as dc
dc.__version__

## Featurizing the PDBBind Dataset


* **voxelization**, a voxel is the 3D analogue of a
pixel.
* **feature types**, sets the types of biophysical and chemical features that the RdkitGridFeaturizer will attempt to detect in input structure.
*   **sanitize**, featurizer will try to clean up any structures (malformed).
* **flatten**, output like one-dimensional feature vector.

The RdkitGridFeaturizer converts a protein to a voxelized representation for use in extracting useful features (biophysical and fingerprints ECFP or SPLIF).

Chemical fingerprints are vectors of 1s and 0s. Each element of the fingerprint vector indicates the presence or absence of a particular molecular feature, defined by some local arrangement of atoms.

ECFP - Extended-connectivity fingerprints, take molecules of
arbitrary size and convert them into fixed-length vectors. 




Note: 1 angstrom is 10^−10 meters

In [2]:
grid_featurizer = dc.feat.RdkitGridFeaturizer(
  voxel_width=2.0, # 2 angstroms
  feature_types=['hbond', 'salt_bridge', 'pi_stack', 'cation_pi', 'ecfp', 'splif'],
  sanitize=True,
  flatten=True
)

## Loding PDBBind dataset

The PDBBind dataset includes experimental binding affinity data and structures for 4852 protein-ligand complexes from the "refined set" and 12800 complexes from the "general set" in PDBBind v2019 and 193 complexes from the "core set" in PDBBind v2013.

**Note: It's necessary to use a GPU. If you are using Google Colab, please change runtime type to GPU.**

In [ ]:
# This will take about 12 minutes
tasks, datasets, transformers = dc.molnet.load_pdbbind(
  featurizer=grid_featurizer,
  splitter='random',
  set_name='core'
)
train_dataset, valid_dataset, test_dataset = datasets

## Building a machine learning model ([random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)) in order to predict.

Random Forest (rf) is used like baseline to compare with a deep model (later we will construc it).

In [4]:
from sklearn.ensemble import RandomForestRegressor

In [5]:
rf_sklearn_model = RandomForestRegressor(n_estimators=100)
rf_model = dc.models.SklearnModel(rf_sklearn_model)

### Training

In [6]:
rf_model.fit(train_dataset)

## Building a deep learning model (neural network) in order to predict

Neural network with three hidden layers, 2048, 1024 and 500 units respectively.

In [23]:
n_features = train_dataset.X.shape[1]
nn_model = dc.models.MultitaskRegressor(
  n_tasks=len(tasks),
  n_features=n_features,
  layer_sizes=[2048, 1024, 500],
  dropout=0.2,
  learning_rate=0.0001
)

### Training

In [24]:
nn_model.fit(train_dataset, nb_epoch=500)

0.10815295219421386

## Checking accuracy of the models

In [25]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

In [26]:
print("Evaluating trained random forest model")
rf_train_scores = rf_model.evaluate(train_dataset, [metric], transformers)
rf_test_scores = rf_model.evaluate(test_dataset, [metric], transformers)
print("Train score: {}".format(rf_train_scores))
print("Test score: {}".format(rf_test_scores))
print("\n")
print("Evaluating trained neural network model")
nn_train_scores = nn_model.evaluate(train_dataset, [metric], transformers)
nn_test_scores = nn_model.evaluate(test_dataset, [metric], transformers)
print("Train score: {}".format(nn_train_scores))
print("Test score: {}".format(nn_test_scores))

Evaluating trained random forest model
Train score: {'pearson_r2_score': 0.8720843042880477}
Test score: {'pearson_r2_score': 0.07945779563998752}


Evaluating trained neural network model
Train score: {'pearson_r2_score': 0.8946227809735714}
Test score: {'pearson_r2_score': 0.2990106345324788}


Note: The performance in test dataset is badly poor, so it's is necessary tunnin in order to improve it.